In [1]:
import pandas as pd
import duckdb
from pathlib import Path
import os
from tqdm.notebook import tqdm


In [2]:
data_dir = Path("/Users/timkeller/git_repos/TK5/Data/nfl-big-data-bowl-2023")

In [3]:
con = duckdb.connect(":memory:")

In [10]:
con.sql("""CREATE TABLE tracking (
        gameId BIGINT, 
        playId BIGINT,  
        nflId BIGINT,
        frameId BIGINT, 
        time TIMESTAMP, 
        jerseyNumber BIGINT,
        team VARCHAR,
        playDirection VARCHAR, 
        x DOUBLE, 
        y DOUBLE, 
        s DOUBLE, 
        a DOUBLE, 
        dis DOUBLE, 
        o DOUBLE,
        dir DOUBLE,
        event VARCHAR, 
        week BIGINT
        );""")

In [14]:
for i in tqdm(range(1,9)):
    con.sql("""
        CREATE TEMP TABLE wk AS 
        SELECT 
        gameId, 
        playId,  
        CASE WHEN nflId = 'NA' THEN NULL ELSE CAST(nflId AS BIGINT) END AS nflId,
        frameId, 
        time, 
        CASE WHEN jerseyNumber = 'NA' THEN NULL ELSE CAST(jerseyNumber AS BIGINT) END AS jerseyNumber,
        team,
        playDirection, 
        x, 
        y, 
        s, 
        a, 
        dis, 
        CASE WHEN o = 'NA' THEN NULL ELSE CAST(o AS DOUBLE) END AS o,
        CASE WHEN dir = 'NA' THEN NULL ELSE CAST(dir AS DOUBLE) END AS dir,
        event, 
        {} as week
        FROM read_csv('{}/week{}.csv')
        """.format(i, str(data_dir), i))
    con.sql("INSERT INTO tracking SELECT * FROM wk")
    con.sql("DROP TABLE wk")

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# import pandas as pd

# weekly_data = pd.DataFrame()

# # Standardize Data if you haven't written a file or don't want to 
# for i in range(1, 10):
#     wk = pd.read_csv(f"tracking_week_{i}.csv")
    
#     wk['x'] = wk.apply(lambda row: 120 - row['x'] if row['playDirection'] == 'left' else row['x'], axis=1)
#     wk['y'] = wk.apply(lambda row: 160 / 3 - row['y'] if row['playDirection'] == 'left' else row['y'], axis=1)
#     wk['dir'] = wk.apply(lambda row: row['dir'] + 180 if row['playDirection'] == 'left' else row['dir'], axis=1)
#     wk['dir'] = wk.apply(lambda row: row['dir'] - 360 if row['dir'] > 360 else row['dir'], axis=1)
#     wk['o'] = wk.apply(lambda row: row['o'] + 180 if row['playDirection'] == 'left' else row['o'], axis=1)
#     wk['o'] = wk.apply(lambda row: row['o'] - 360 if row['o'] > 360 else row['o'], axis=1)
#     wk['second'] = wk['frameId'] / 10
    
#     wk['week'] = i
#     weekly_data = pd.concat([weekly_data, wk], ignore_index=True)

In [15]:
con.execute("""
    CREATE OR REPLACE TABLE tracking_std AS
    SELECT *,
            CASE WHEN playDirection = 'left' THEN 120 - x ELSE x END AS x,
            CASE WHEN playDirection = 'left' THEN 160 / 3 - y ELSE y END AS y,
            CASE WHEN playDirection = 'left' THEN dir + 180 ELSE dir END AS dir,
            CASE WHEN dir > 360 THEN dir - 360 ELSE dir END AS dir,
            CASE WHEN playDirection = 'left' THEN o + 180 ELSE o END AS o,
            CASE WHEN o > 360 THEN o - 360 ELSE o END AS o,
            frameId / 10 AS second,
    FROM tracking
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.

In [38]:
con.sql("DROP TABLE tracking;")

In [16]:
con.sql("DESCRIBE tracking_std;")

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │ column_type │  null   │   key   │ default │  extra  │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ gameId        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ playId        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ nflId         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ frameId       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ time          │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ jerseyNumber  │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ team          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ playDirection │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ x             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ y             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NU

In [28]:
con.sql("SELECT * FROM tracking LIMIT 10").df()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511,1,2021-09-10 00:26:31,12,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511,2,2021-09-10 00:26:31,12,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511,3,2021-09-10 00:26:31,12,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511,4,2021-09-10 00:26:31,12,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511,5,2021-09-10 00:26:31,12,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None
5,2021090900,97,25511,6,2021-09-10 00:26:31,12,TB,right,37.64,24.26,0.35,0.53,0.05,144.42,282.72,ball_snap
6,2021090900,97,25511,7,2021-09-10 00:26:31,12,TB,right,37.56,24.26,0.54,1.05,0.08,137.49,272.95,None
7,2021090900,97,25511,8,2021-09-10 00:26:31,12,TB,right,37.47,24.25,0.80,1.85,0.09,131.95,267.49,None
8,2021090900,97,25511,9,2021-09-10 00:26:31,12,TB,right,37.38,24.24,0.99,2.03,0.09,129.85,263.48,None
9,2021090900,97,25511,10,2021-09-10 00:26:32,12,TB,right,37.27,24.23,1.19,1.82,0.11,123.79,263.77,None


In [26]:
con.sql("""    
        CREATE OR REPLACE TABLE tracking_wk1 AS 
        SELECT 
        gameId, 
        playId,  
        CASE WHEN nflId = 'NA' THEN NULL ELSE CAST(nflId AS BIGINT) END AS nflId,
        frameId, 
        time, 
        CASE WHEN jerseyNumber = 'NA' THEN NULL ELSE CAST(jerseyNumber AS BIGINT) END AS jerseyNumber,
        team,
        playDirection, 
        x, 
        y, 
        s, 
        a, 
        dis, 
        CASE WHEN o = 'NA' THEN NULL ELSE CAST(o AS DOUBLE) END AS o,
        CASE WHEN dir = 'NA' THEN NULL ELSE CAST(dir AS DOUBLE) END AS dir,
        event, 
        1 as week
        FROM read_csv('{}/week1.csv')
                 """.format(str(data_dir)))

In [27]:
df_wk1 = con.sql("SELECT * FROM tracking_wk1").df()

In [ ]:
CASE WHEN dir > 360 THEN dir - 360 ELSE dir END AS dir,        


In [29]:
df_wk1_tst = con.sql("""
        SELECT 
        gameId, 
        playId,  
        nflId,
        frameId, 
        time, 
        jerseyNumber,
        team,
        playDirection, 
        CASE WHEN playDirection = 'left' THEN 120 - x ELSE x END AS x,
        CASE WHEN playDirection = 'left' THEN 160 / 3 - y ELSE y END AS y,
        s, 
        a, 
        dis, 
        o,
        CASE WHEN playDirection = 'left' THEN dir + 180 WHEN dir > 360 THEN dir - 360 ELSE dir END AS dir,
        event, 
        week
        FROM tracking_wk1
        """).df()

In [31]:
df_wk1_tst['dir'].max()

np.float64(540.0)